In [4]:
import numpy as np
import pandas as pd
import scipy
import scipy.optimize as optim
from helpers import *
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import os

In [5]:
def ZemelTransform(encoded_data):
    """
    This function learns the model from the training set for Adult data which is binarized (in the steps below) and passed to the LBFGS solver
    after computing the composite function using helpers.py. helpers.py has all the functions used in Zemel's paper as a s
    separate module. As given in the paper the parameter setting (0.01,1,50) with
    a k=10 gives good accuracy and reasonable discrimination for Adult dataset. Another setting of (0.01,0.01,0) gives low
    discrimination. k value can be modified in the code below or passed as a parameter.
    """
    k=5
    df_train=encoded_data
    catlabelsage=pd.get_dummies(encoded_data['Age (decade)'])
    catlabelseducation=pd.get_dummies(encoded_data['Education Years'])
    encoded_data=pd.concat([encoded_data,catlabelsage],axis=1)
    encoded_data=encoded_data.drop(['Age (decade)'],axis=1)
    encoded_data=pd.concat([encoded_data,catlabelseducation],axis=1)
    encoded_data=encoded_data.drop(['Education Years'],axis=1)
    encoded_data=encoded_data.drop(['Income'],axis=1)
    encoders = {}
    encoders['Gender'] = preprocessing.LabelEncoder()
    encoded_data['Gender'] = encoders['Gender'].fit_transform(encoded_data['Gender'])
    sensitive_idx = np.array(np.where(encoded_data['Gender']==0))[0].flatten()
    nonsensitive_idx = np.array(np.where(encoded_data['Gender']!=0))[0].flatten()
    original_sensitive=df_train.iloc[sensitive_idx,:]
    original_nonsensitive=df_train.iloc[nonsensitive_idx,:]
    originaldat=pd.concat((original_sensitive,original_nonsensitive))
    ytrain_sensitive=np.array(encoded_data['Income Binary'])[sensitive_idx]
    ytrain_nonsensitive=np.array(encoded_data['Income Binary'])[nonsensitive_idx]
    encoded_data=encoded_data.drop('Income Binary',axis=1)
    traindat=np.array(encoded_data)
    training_sensitive=traindat[sensitive_idx,:]
    training_nonsensitive=traindat[nonsensitive_idx,:]
    rez = np.random.uniform(size=traindat.shape[1] * 2 + k + traindat.shape[1] * k)
    bnd = []
    for i, k2 in enumerate(rez):
        if i < traindat.shape[1] * 2 or i >= traindat.shape[1] * 2 + k:
            bnd.append((None, None))
        else:
            bnd.append((0, 1))
    rez = optim.fmin_l_bfgs_b(LFR, x0=rez, epsilon=1e-5, 
                          args=(training_sensitive, training_nonsensitive, 
                                ytrain_sensitive, ytrain_nonsensitive, k, 0.01,
                                1, 50, 0),
                          bounds = bnd, approx_grad=True, maxfun=15000, 
                          maxiter=15000)
    
    # extract values from obtained result vector
    Ns, P = training_sensitive.shape
    N,_=training_nonsensitive.shape
    # alpha values for sensitive (0 after sklearn label encoder) and non-sensitive (1) groups.
    alphaoptim0 = rez[0][:P]
    alphaoptim1 = rez[0][P : 2 * P]
    # weight vector
    woptim = rez[0][2 * P : (2 * P) + k]
    #cluster representatives
    voptim = np.matrix(rez[0][(2 * P) + k:]).reshape((k, P))
    
    # learned distance function
    dist_sensitive=distances(training_sensitive, voptim, alphaoptim1, Ns, P, k)
    dist_nonsensitive=distances(training_nonsensitive,voptim,alphaoptim0,N,P,k)
    
    # learned cluster mapping probabilities
    M_nk_sensitive=M_nk(dist_sensitive, Ns, k)
    M_nk_nonsensitive=M_nk(dist_nonsensitive,N,k)
    
    # learned mappings
    res_sensitive=x_n_hat(training_sensitive, M_nk_sensitive, voptim, Ns, P, k)
    x_n_hat_sensitive=res_sensitive[0]
    res_nonsensitive=x_n_hat(training_nonsensitive, M_nk_nonsensitive, voptim, N, P, k)
    x_n_hat_nonsensitive=res_nonsensitive[0]
    
    # compute predictions for training dataset
    res_sensitive=yhat(M_nk_sensitive, ytrain_sensitive, woptim, Ns, k)
    y_hat_sensitive=res_sensitive[0]
    res_nonsensitive=yhat(M_nk_nonsensitive, ytrain_nonsensitive, woptim, N, k)
    y_hat_nonsensitive=res_nonsensitive[0]
    
    # preserve ordering (done here because of how the implementation is to minimize confusion for me!)
    yordered=np.concatenate((ytrain_sensitive,ytrain_nonsensitive))
    #ordered final train predictions
    yhatordered=np.concatenate((y_hat_sensitive,y_hat_nonsensitive))
    
    traindataordered=np.concatenate((training_sensitive,training_nonsensitive))
    # ordered final train mappings
    traindatahatordered=np.concatenate((x_n_hat_sensitive,x_n_hat_nonsensitive))
        
    return originaldat,traindataordered,yordered,traindatahatordered,yhatordered,rez

def ZemelPrediction(testing_sensitive,testing_nonsensitive,ytest_sensitive,ytest_nonsensitive,rez):
    """
    This code uses the model learned from the training data and applies that model (rez) on the test data to obtain the final
    yhat predictions (yhat=ytilde here as there is no separate prediction algorithm applied here). The resulting predictions
    on test are returned.
    """
    k=5
    # extract training model parameters
    Ns, P = testing_sensitive.shape
    N,_=testing_nonsensitive.shape
    alphaoptim0 = rez[0][:P]
    alphaoptim1 = rez[0][P : 2 * P]
    woptim = rez[0][2 * P : (2 * P) + k]
    voptim = np.matrix(rez[0][(2 * P) + k:]).reshape((k, P))
    
    # compute distances on the test dataset using train model params
    dist_sensitive=distances(testing_sensitive, voptim, alphaoptim1, Ns, P, k)
    dist_nonsensitive=distances(testing_nonsensitive,voptim,alphaoptim0,N,P,k)
    
    #compute cluster probabilities for test instances
    M_nk_sensitive=M_nk(dist_sensitive, Ns, k)
    M_nk_nonsensitive=M_nk(dist_nonsensitive,N,k)
    
    #compute predictions for test instances
    res_sensitive=yhat(M_nk_sensitive, ytest_sensitive, woptim, Ns, k)
    y_hat_sensitive=res_sensitive[0]
    res_nonsensitive=yhat(M_nk_nonsensitive, ytest_nonsensitive, woptim, N, k)
    y_hat_nonsensitive=res_nonsensitive[0]
    
    # return ordered test response and test predictions 
    yordered=np.concatenate((ytest_sensitive,ytest_nonsensitive))
    yhatordered=np.concatenate((y_hat_sensitive,y_hat_nonsensitive))
    return yordered,yhatordered

def ZemelTestPreprocessing(df_test_new):
    """
    This code uses the model learned from the training data and applies that model (rez) on the test data to obtain the final
    yhat predictions (yhat=ytilde here as there is no separate prediction algorithm applied here). The resulting predictions
    on test are returned.
    """
    encoded_data=df_test_new
    catlabelsage=pd.get_dummies(encoded_data['Age (decade)'])
    catlabelseducation=pd.get_dummies(encoded_data['Education Years'])
    encoded_data=pd.concat([encoded_data,catlabelsage],axis=1)
    encoded_data=encoded_data.drop(['Age (decade)'],axis=1)
    encoded_data=pd.concat([encoded_data,catlabelseducation],axis=1)
    encoded_data=encoded_data.drop(['Education Years'],axis=1)
    encoded_data=encoded_data.drop(['Income'],axis=1)
    encoders = {}
    encoders['Gender'] = preprocessing.LabelEncoder()
    encoded_data['Gender'] = encoders['Gender'].fit_transform(encoded_data['Gender'])
    sensitive_idx = np.array(np.where(encoded_data['Gender']==0))[0].flatten()
    nonsensitive_idx = np.array(np.where(encoded_data['Gender']!=0))[0].flatten()
    original_sensitive=df_test_new.iloc[sensitive_idx,:]
    original_nonsensitive=df_test_new.iloc[nonsensitive_idx,:]
    originaldat=pd.concat((original_sensitive,original_nonsensitive))
    ytest_sensitive=np.array(encoded_data['Income Binary'])[sensitive_idx]
    ytest_nonsensitive=np.array(encoded_data['Income Binary'])[nonsensitive_idx]
    encoded_data=encoded_data.drop('Income Binary',axis=1)
    testdat=np.array(encoded_data)
    testing_sensitive=testdat[sensitive_idx,:]
    testing_nonsensitive=testdat[nonsensitive_idx,:]
    return testing_sensitive,testing_nonsensitive,ytest_sensitive,ytest_nonsensitive,originaldat

In [6]:
path =r'../experiment_data1/'
#These are the test and train datasets by splitting the original data.
train_0 = pd.read_csv(path + "train_0.csv",index_col=None, header=0, usecols=range(1,6))
test_0 = pd.read_csv(path + "test_0.csv",index_col=None, header=0, usecols=range(1,6))

train_0.shape

(26048, 5)

In [7]:
zemel_df_train_res=ZemelTransform(train_0)

/home/bhanu/anaconda3/envs/python2/lib/python2.7/site-packages/numba/dataflow.py:346: RuntimeWarning: Python2 style print partially supported.  Please use Python3 style print.
  "Python3 style print.", RuntimeWarning)


(250, 17308.859644389097)

(500, 15236.493871300623)

(750, 13803.67692730774)

(1000, 13614.82064304608)

(1250, 12739.54254332751)

(1500, 12446.45174810471)

(1750, 12302.737130250951)

(2000, 12057.127614571888)

(2250, 11986.998719936026)

(2500, 11837.189582599587)

(2750, 11778.405433248709)

(3000, 11731.132897569007)

(3250, 11673.71556016283)

(3500, 11661.371236842282)

(3750, 11635.591178552817)

(4000, 11593.64605763025)

(4250, 11577.474722048915)

(4500, 11566.815507085354)

(4750, 11556.168273097446)

(5000, 11540.495084023085)

(5250, 11530.69437128558)

(5500, 11528.589537316311)

(5750, 11521.349653729309)

(6000, 11517.110823428247)

(6250, 11511.087930392725)

(6500, 11502.595023811515)

(6750, 11500.113926863254)

(7000, 11498.63749901459)

(7250, 11497.141958848173)

(7500, 11494.21346902748)

(7750, 11490.303127262248)

(8000, 11485.994663410742)

(8250, 11484.486876576677)

(8500, 11482.92312941706)

(8750, 11480.332966775564)

(9000, 11478.921576802615)

(9250

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from __future__ import division

print '----------------------------------------------------------------'
print 'Zemel algorithm (ICML 2013) performance:'
response=zemel_df_train_res[2]
pred=zemel_df_train_res[4]
print 'Train performance with pert. dataset: '
print roc_auc_score(response,pred)

zemel_preprocess=ZemelTestPreprocessing(test_0)
zemel_prediction= ZemelPrediction(zemel_preprocess[0],zemel_preprocess[1],zemel_preprocess[2],zemel_preprocess[3],zemel_df_train_res[5])
print 'Perturbed test performance when scored on original test y variable: '
print roc_auc_score(zemel_prediction[0],zemel_prediction[1])

# save performance
df_test_pred = zemel_preprocess[4]
df_test_pred['pred'] = zemel_prediction[1]

mean = df_test_pred.groupby('Gender')['pred'].mean()
v = mean.values
v = v.reshape(len(v),1)
ratio_df = pd.DataFrame(v/v.transpose(),index=mean.index,columns=mean.index )
ratio_df_arr=np.asarray(np.abs(1-ratio_df))
zemel_discrim=np.amax(ratio_df_arr)
zemel_discrim

----------------------------------------------------------------
Zemel algorithm (ICML 2013) performance:
Train performance with pert. dataset: 
0.821456810411
Perturbed test performance when scored on original test y variable: 
0.818830165357


1.6781477431939105

In [6]:
mean = df_test_pred.groupby('Gender')['pred'].mean()

In [10]:
zemel_preprocess[1].shape

(4372, 17)